In [10]:
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma -q




In [11]:
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud

In [12]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_3')


In [13]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/proj.json"


step:1 pdf loading.

In [14]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/somatosensory.pdf")
data = loader.load()


In [15]:
len(data)

4

step:2 splitting the data of pdf file.

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

total number of documents :  13


step:4 creating embedding of the data.

In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma


In [18]:
embedding_Model = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)

step:5 Embedding data and retriver.

In [19]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_Model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})



step:6 creating llm model.

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.3 , max_tokens=1000)

step:7 defining user prompt.

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

step:8 creating chain

In [22]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

step:9 asking question not included in pdf.

In [23]:
response = rag_chain.invoke({"input" : "what is k2"})
print(response["answer"])

I'm sorry, but the provided text does not contain any information about "k2".


step:10 asking questions included in pdf .

In [24]:
response = rag_chain.invoke({"input" : "what is somatosensory"})
print(response["answer"])

The somatosensory system consists of sensors in the skin and sensors in our muscles, tendons, and joints. The receptors in the skin tell us about temperature, pressure and surface texture, and pain. The receptors in muscles and joints provide information about muscle length, muscle tension, and joint angles.
